# 환경설정

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# 1. 문서로드

In [2]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("data/SPRi AI Brief_10월호_산업동향_F.pdf")

docs = loader.load()

print(f"문서의 페이지수 : {len(docs)}")

문서의 페이지수 : 25


# 2. 문서 분할

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의 수 : {len(split_documents)}")

분할된 청크의 수 : 499


# 3. 임베딩 생성

In [7]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

# 4. DB생성

In [12]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [13]:
for doc in vectorstore.similarity_search("딥페이크"):
    print(doc.page_content)

n 일부 의원은 딥페이크 음란물의 제작이 처벌 범위에 빠진 점을 지적하는 한편, 선거 관련 
딥페이크에 대한 처벌 요구도 제기 
KEY Contents
계획을 수립
£ 딥페이크 음란물 제작 및 선거 영역까지 처벌 범위를 확대해야 한다는 지적도 제기
일각에서는 법안이 불충분하며 동의 없는 딥페이크 음란물의 제작 및 이를 제작하겠다는 위협도 
범죄로 간주해야 한다고 주장
n 한편, 데이비드 포콕(David Pocock) 무소속 상원 의원은 딥페이크 처벌 범위가 선거와 관련된 영역까지 
확대되어야 한다고 지적


# 5. 검색기 (Retriever) 생성

In [14]:
retriever = vectorstore.as_retriever()

In [15]:
retriever.invoke("딥페이크가 뭐야?")

[Document(metadata={'source': 'data/SPRi AI Brief_10월호_산업동향_F.pdf', 'file_path': 'data/SPRi AI Brief_10월호_산업동향_F.pdf', 'page': 5, 'total_pages': 25, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13947', 'producer': 'Hancom PDF 1.3.0.547', 'creationDate': "D:20241007090546+09'00'", 'modDate': "D:20241007090546+09'00'", 'trapped': ''}, page_content='n 일부 의원은 딥페이크 음란물의 제작이 처벌 범위에 빠진 점을 지적하는 한편, 선거 관련 \n딥페이크에 대한 처벌 요구도 제기 \nKEY Contents'),
 Document(metadata={'source': 'data/SPRi AI Brief_10월호_산업동향_F.pdf', 'file_path': 'data/SPRi AI Brief_10월호_산업동향_F.pdf', 'page': 5, 'total_pages': 25, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13947', 'producer': 'Hancom PDF 1.3.0.547', 'creationDate': "D:20241007090546+09'00'", 'modDate': "D:20241007090546+09'00'", 'trapped': ''}, page_content='일각에서는 법안이 불충분하며 동의 없는 딥페이크 음란물의 제작 및 이를 제작하겠다는 위협도 \n범죄로 간주해야 한다고 주장'),
 Document(me

# 6. 프롬프트 생성

In [17]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question.
Use a very kind and gentle tone like a kindergarten teacher talking to a child.
Speak in a warm and friendly way.
If you don't know the answer, just say that you don't know. 
Answer in Korean. 

#Context:
{context}

#Question
{question}

#Answer:"""


)

# 7. LLM모델 생성

In [19]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 8. Chain생성

In [20]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context" : retriever, "question" : RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

In [21]:
question = "딥페이크에 문제점은 뭐야?"
response = chain.invoke(question)

print(response)

딥페이크는 아주 재미있는 기술이지만, 몇 가지 문제점이 있어요. 딥페이크를 다른 사람의 동의 없이 사용하면 민주주의를 위협할 수 있대요. 그래서 선거를 보호하기 위해 딥페이크를 금지해야 한다고 해요. 또, 딥페이크 음란물을 만드는 것도 큰 문제인데, 이런 행동은 처벌받아야 한다고 많은 사람들이 말하고 있어요. 딥페이크를 잘못 사용하면 다른 사람에게 큰 피해를 줄 수 있으니 조심해야 해요.
